In [1]:
from langchain_groq import ChatGroq

In [2]:
llm=ChatGroq(
    temperature=0,
    groq_api_key='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',
    model_name="llama-3.3-70b-versatile"   
)
response=llm.invoke("Who is prime minister of India ?")
print(response.content)

The current Prime Minister of India is Narendra Modi. He has been serving as the Prime Minister of India since May 26, 2014.


In [3]:
# %pip install langchain-community beautifulsoup4

In [4]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://www.amazon.jobs/en/jobs/3063354/software-dev-engineer-ii-cross-border-seller-programs")
page_data=loader.load().pop().page_content
print(page_data)

USER_AGENT environment variable not set, consider setting it to identify your requests.


Software Dev Engineer II, Cross Border Seller Programs - Job ID: 3063354 | Amazon.jobs
Skip to main contentHomeTeamsLocationsJob categoriesMy careerMy applicationsMy profileAccount securitySettingsSign outResourcesDisability accommodationsBenefitsInclusive experiencesInterview tipsLeadership principlesWorking at AmazonFAQ×Software Dev Engineer II, Cross Border Seller ProgramsJob ID: 3063354 | ADCI - KarnatakaApply nowDESCRIPTIONStay local, shop global! Amazon Cross Border (XB) team allows customers to easily discover a wide selection of international products available from Amazon’s websites around the world, offering simplified customs, faster delivery, local language and payment support, and a world-class customer service. XB team has presence in multiple countries and will continue to stay at the forefront of Amazon’s global expansion strategy. XB Seller Experience is a key driver for the growth for this business and we are looking for a dynamic, organized and self-starting SDEs to 

In [5]:
from langchain_core.prompts import PromptTemplate

prompt_extract= PromptTemplate.from_template(
    """
    ### SCRAPED TEXT FROM WEBSITE:
    {page_data}
    ### INSTRUCTION:
    The scraped text is from the career's page of website.
    Your job is to extract the job postingd and return them in JSON format containing the
    following keys: 'role'. 'experience', 'skills' and 'description'.
    Only return the valid JSON.
    ## VALID JSON (NO PREAMBLE):
    """
    
)
chain_extract = prompt_extract | llm
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [6]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Software Dev Engineer II, Cross Border Seller Programs',
 'experience': '3+ years of non-internship professional software development experience',
 'skills': ['Programming with at least one software programming language',
  'Design or architecture (design patterns, reliability and scaling) of new and existing systems',
  'Full software development life cycle, including coding standards, code reviews, source control management, build processes, testing, and operations'],
 'description': 'The XB Seller Programs Team is seeking an experienced Software Development Engineer interested in disrupting the way products are made available to customers, sold, and distributed on a global scale. The right candidate will have hands-on involvement in building new tools, services, and distribution structures across Amazon’s growing footprint.'}

In [7]:
type(json_res)

dict

In [8]:
# !pip install pandas

In [9]:
import pandas as pd

df=pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [12]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if  not collection.count():
    for _,row in df.iterrows():
        collection.add(documents=row["Techstack"],
                      metadatas={"links":row["Links"]},
                      ids=[str(uuid.uuid4())])

In [16]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', []) 
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/android-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/java-portfolio'}]]

In [15]:
job = json_res
job['skills']

['Programming with at least one software programming language',
 'Design or architecture (design patterns, reliability and scaling) of new and existing systems',
 'Full software development life cycle, including coding standards, code reviews, source control management, build processes, testing, and operations']

In [17]:
prompt_email_generic = PromptTemplate.from_template(
    """
    ### JOB DESCRIPTION:
    {job_description}
    
    ### INSTRUCTION:
    You are Mohan, a Business Development Executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to 
    facilitating the seamless integration of business processes through automated tools. 
    We specialize in delivering scalable solutions, AI-driven platforms, and advanced software engineering tailored 
    to client needs. 
    Your job is to write a cold email to the concerned person regarding the job mentioned above, 
    describing how AtliQ’s expertise can fulfill their requirements. 
    
    Also, add the most relevant ones from the following links to showcase AtliQ’s portfolio: {link_list}
    
    Remember you are Mohan, BDE at AtliQ. 
    Do not provide a preamble. 
    
    ### EMAIL (NO PREAMBLE):
    
    """
)

chain_email_generic = prompt_email_generic | llm
res = chain_email_generic.invoke({"job_description": str(job), "link_list": links})
print(res.content)


Subject: Expert Software Development Solutions for Cross Border Seller Programs

Dear Hiring Manager,

I came across the job description for a Software Dev Engineer II, Cross Border Seller Programs, and I was impressed by the scope of the project. As a Business Development Executive at AtliQ, an AI & Software Consulting company, I believe our expertise can fulfill your requirements and contribute to the success of your team.

With 3+ years of experience in software development, our team possesses the necessary skills to design, architect, and develop new and existing systems, ensuring reliability and scalability. We have hands-on experience with the full software development life cycle, including coding standards, code reviews, source control management, build processes, testing, and operations. Our proficiency in programming languages, such as Python, Java, and others, enables us to tackle complex projects and deliver high-quality solutions.

At AtliQ, we specialize in delivering scal